python3 -m pip install -U git+https://github.com/adefossez/demucs#egg=demucs

In [1]:
import kagglehub

# Download latest version
path = kagglehub.model_download("google/yamnet/tensorFlow2/yamnet")

print("Path to model files:", path)

Path to model files: /Users/andrewchang/.cache/kagglehub/models/google/yamnet/tensorFlow2/yamnet/1


In [10]:
model = tf.saved_model.load(
    path, tags=None, options=None
)

In [13]:
model.signatures

_SignatureMap({'serving_default': <ConcreteFunction (*, waveform: TensorSpec(shape=(None,), dtype=tf.float32, name='waveform')) -> Dict[['output_1', TensorSpec(shape=(None, 1024), dtype=tf.float32, name='output_1')], ['output_0', TensorSpec(shape=(None, 521), dtype=tf.float32, name='output_0')], ['output_2', TensorSpec(shape=(None, 64), dtype=tf.float32, name='output_2')]] at 0x16AE5DFD0>})

In [2]:
import demucs.api
import soundfile as sf
import torchaudio
import torch
import scipy.io as sio
import tensorflow as tf
# import tensorflow_hub as hub

/Users/andrewchang/opt/anaconda3/envs/demucs/lib/python3.9/site-packages/demucs/api.py:27: UserWarning: TorchAudio's global backend is now deprecated. Please enable distpatcher by setting `TORCHAUDIO_USE_BACKEND_DISPATCHER=1`, and specify backend when calling load/info/save function.
  import torchaudio as ta
2024-04-05 14:29:31.337863: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Initialize with default parameters:
separator = demucs.api.Separator()

In [4]:
# import librosa
# y, sr = librosa.load('/Users/andrewchang/NYU_research/MusicSpeech-STM/data/musicCorp/fma_large/000/000002.mp3')
# y, sr = sf.read('/Users/andrewchang/NYU_research/MusicSpeech-STM/data/musicCorp/fma_large/000/000002.mp3')
y, sr = torchaudio.load('/Users/andrewchang/NYU_research/MusicSpeech-STM/data/musicCorp/fma_large/000/000002.mp3')


In [5]:
y.shape

torch.Size([2, 1321344])

In [6]:
sr

44100

In [7]:
y_tensor = y

In [8]:
# import numpy as np
# import torch
# # y_2 = np.stack((y, y))
# y_tensor = torch.tensor(y)

# if y_tensor.shape[0] > y_tensor.shape[1]:
#     y_tensor = y_tensor.T

# Duplicate and stack the tensor
if y.shape[0]==1:
    y_tensor = torch.stack((y_tensor, y_tensor), dim=0)

y_tensor.shape

torch.Size([2, 1321344])

In [9]:
origin, separated = separator.separate_tensor(wav=y_tensor, sr=sr)


In [9]:
separated

{'drums': tensor([[2.3971e-04, 1.6992e-04, 2.2337e-04,  ..., 2.0685e-03, 1.6706e-03,
          1.4716e-03],
         [1.7512e-04, 8.9158e-05, 1.3772e-04,  ..., 2.1505e-03, 1.0183e-03,
          2.5540e-03]]),
 'bass': tensor([[0.0003, 0.0003, 0.0003,  ..., 0.0154, 0.0153, 0.0151],
         [0.0003, 0.0003, 0.0003,  ..., 0.0177, 0.0175, 0.0174]]),
 'other': tensor([[ 0.0001,  0.0002,  0.0001,  ..., -0.0075, -0.0121, -0.0134],
         [ 0.0004,  0.0005,  0.0003,  ..., -0.0020, -0.0127, -0.0117]]),
 'vocals': tensor([[1.0755e-04, 1.8422e-04, 2.3100e-04,  ..., 9.5913e-02, 7.9966e-02,
          4.2849e-02],
         [2.0579e-04, 2.3162e-04, 3.1246e-04,  ..., 1.1696e-01, 1.5019e-01,
          1.3584e-01]])}

In [14]:
separated['vocals'].shape

torch.Size([2, 1321344])

In [19]:
separated['vocals'].mean(axis=0)

tensor([0.0003, 0.0003, 0.0003,  ..., 0.1015, 0.1110, 0.0845])

In [20]:
import numpy as np
scores, embeddings, spectrogram = model(np.array(separated['vocals'].mean(axis=0)))

In [21]:
scores

<tf.Tensor: shape=(172, 521), dtype=float32, numpy=
array([[4.5364147e-01, 4.5595123e-03, 1.1929839e-03, ..., 2.7698800e-03,
        1.2314862e-03, 9.1753968e-08],
       [1.6486099e-01, 1.3112247e-02, 1.0731223e-03, ..., 3.2815991e-03,
        3.9036016e-03, 6.9473472e-08],
       [3.8418364e-02, 1.4409499e-03, 1.1317354e-03, ..., 5.7045547e-03,
        4.5717005e-03, 7.2389389e-07],
       ...,
       [5.4133873e-02, 6.0582132e-04, 3.6296496e-04, ..., 1.7845613e-03,
        1.0789689e-03, 9.4569430e-07],
       [6.6219610e-03, 3.1086968e-03, 4.8757669e-05, ..., 4.1922869e-04,
        1.9914478e-04, 1.6626602e-08],
       [8.1586674e-02, 3.1353948e-03, 3.2573056e-03, ..., 2.8697215e-02,
        9.8005384e-03, 1.2756771e-04]], dtype=float32)>

In [22]:
scores_np = scores.numpy()
spectrogram_np = spectrogram.numpy()
infered_class = class_names[scores_np.mean(axis=0).argmax()]
print(f'The main sound is: {infered_class}')

NameError: name 'class_names' is not defined

In [23]:
scores_np

array([[4.5364147e-01, 4.5595123e-03, 1.1929839e-03, ..., 2.7698800e-03,
        1.2314862e-03, 9.1753968e-08],
       [1.6486099e-01, 1.3112247e-02, 1.0731223e-03, ..., 3.2815991e-03,
        3.9036016e-03, 6.9473472e-08],
       [3.8418364e-02, 1.4409499e-03, 1.1317354e-03, ..., 5.7045547e-03,
        4.5717005e-03, 7.2389389e-07],
       ...,
       [5.4133873e-02, 6.0582132e-04, 3.6296496e-04, ..., 1.7845613e-03,
        1.0789689e-03, 9.4569430e-07],
       [6.6219610e-03, 3.1086968e-03, 4.8757669e-05, ..., 4.1922869e-04,
        1.9914478e-04, 1.6626602e-08],
       [8.1586674e-02, 3.1353948e-03, 3.2573056e-03, ..., 2.8697215e-02,
        9.8005384e-03, 1.2756771e-04]], dtype=float32)